In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import warnings

In [9]:
train = pd.read_csv('prediction_log/train_log.csv')
test = pd.read_csv('prediction_log/test_log.csv')

In [10]:
new_train = train
new_test = test

In [11]:
new_train.drop(columns=['enroll_id', 'object'], inplace=True)
new_test.drop(columns=['enroll_id', 'object'], inplace=True)

In [12]:
# Substitute various actions with category-type actions
video_action = ['seek_video','play_video','pause_video','stop_video','load_video']
problem_action = ['problem_get','problem_check','problem_save','reset_problem','problem_check_correct', 'problem_check_incorrect']
forum_action = ['create_thread','create_comment','delete_thread','delete_comment']
click_action = ['click_info','click_courseware','click_about','click_forum','click_progress']
close_action = ['close_courseware', 'close_forum']

video = ['video']*len(video_action)
problem = ['problem']*len(problem_action)
forum = ['forum']*len(forum_action)
click = ['click']*len(click_action)
close = ['close']*len(close_action)

video_dict = dict(zip(video_action, video))
problem_dict = dict(zip(problem_action, problem))
forum_dict = dict(zip(forum_action, forum))
click_dict = dict(zip(click_action, click))
close_dict = dict(zip(close_action, close))

new_train['action'].replace(video_dict, inplace=True)
new_train['action'].replace(problem_dict, inplace=True)
new_train['action'].replace(forum_dict, inplace=True)
new_train['action'].replace(click_dict, inplace=True)
new_train['action'].replace(close_dict, inplace=True)

new_test['action'].replace(video_dict, inplace=True)
new_test['action'].replace(problem_dict, inplace=True)
new_test['action'].replace(forum_dict, inplace=True)
new_test['action'].replace(click_dict, inplace=True)
new_test['action'].replace(close_dict, inplace=True)

### Retrieve time spent on courses from join with course_info dataframe

In [13]:
courseinfo = pd.read_csv('course_info.csv', index_col='id')

In [14]:
new_train = new_train.merge(courseinfo, on='course_id')
new_test = new_test.merge(courseinfo, on='course_id')

In [15]:
new_train.drop(columns=['end', 'course_type', 'category'], inplace=True)
new_test.drop(columns=['end', 'course_type', 'category'], inplace=True)

In [16]:
new_train['time']= pd.to_datetime(new_train['time'])
new_train['start']= pd.to_datetime(new_train['start'])
new_test['time']= pd.to_datetime(new_test['time'])
new_test['start']= pd.to_datetime(new_test['start'])

In [17]:
new_train['week'] = (new_train['time'] - new_train['start']).div(7)
new_test['week'] = (new_test['time'] - new_test['start']).div(7)
def to_weeks(x):
    return int(str(x)[0]) + 1
new_train['week'] = new_train['week'].transform(to_weeks)
new_test['week'] = new_test['week'].transform(to_weeks)

In [18]:
new_train.drop(columns=['start','time', 'session_id'], inplace=True)
new_test.drop(columns=['start', 'time', 'session_id'], inplace=True)

In [19]:
new_train

,username,course_id,action,week
0,5981,course-v1:TsinghuaX+70800232X+2015_T2,click,1
1,5981,course-v1:TsinghuaX+70800232X+2015_T2,click,1
2,1544995,course-v1:TsinghuaX+70800232X+2015_T2,video,4
3,1544995,course-v1:TsinghuaX+70800232X+2015_T2,video,4
4,1544995,course-v1:TsinghuaX+70800232X+2015_T2,video,4
...,...,...,...,...
29165535,2659552,course-v1:TsinghuaX+AP000001X+2016_T1,click,5
29165536,2659552,course-v1:TsinghuaX+AP000001X+2016_T1,problem,5
29165537,2659552,course-v1:TsinghuaX+AP000001X+2016_T1,problem,5
29165538,2659552,course-v1:TsinghuaX+AP000001X+2016_T1,problem,5


In [20]:
new_train['count'] = len(new_train)*[1]
new_test['count'] = len(new_test)*[1]

In [21]:
new_train = new_train.groupby(by=['username', 'course_id', 'action', 'week']).count().reset_index()
new_test = new_test.groupby(by=['username', 'course_id', 'action', 'week']).count().reset_index()

In [22]:
new_train

,username,course_id,action,week,count
0,5,TsinghuaX/60510102X/_,close,1,1
1,5,TsinghuaX/60510102X/_,video,1,10
2,5,TsinghuaX/70240183x/2015_T2,close,1,3
3,5,TsinghuaX/70240183x/2015_T2,problem,1,1
4,5,TsinghuaX/70240183x/2015_T2,video,1,39
...,...,...,...,...,...
839000,7023562,course-v1:TsinghuaX+AP000008X+2017T1,problem,2,10
839001,7023562,course-v1:TsinghuaX+AP000008X+2017T1,video,2,228
839002,7036067,course-v1:TsinghuaX+AP000008X+2017T1,click,3,7
839003,7036067,course-v1:TsinghuaX+AP000008X+2017T1,problem,3,8


In [23]:
new_test

,username,course_id,action,week,count
0,5,course-v1:Tsinghua+20150001+2015_T2,click,1,14
1,5,course-v1:Tsinghua+20150001+2015_T2,close,1,3
2,5,course-v1:Tsinghua+20150001+2015_T2,video,1,12
3,5,course-v1:TsinghuaX+00740123X+2016_T1,click,5,2
4,5,course-v1:TsinghuaX+00740123X+2016_T1,video,5,5
...,...,...,...,...,...
358951,7006106,course-v1:TsinghuaX+AP000008X+2017T1,video,2,9
358952,7008105,course-v1:TsinghuaX+AP000008X+2017T1,click,1,1
358953,7008105,course-v1:TsinghuaX+AP000008X+2017T1,problem,1,3
358954,7008105,course-v1:TsinghuaX+AP000008X+2017T1,video,1,1


### Save dataframes to disk

In [ ]:
# Save pivot tables
new_train.to_csv('dataset/new_train.csv')
new_test.to_csv('dataset/new_test.csv')

# From now on it is only useful for Prophet

In [24]:
new_train_prophet = new_train.groupby(by=['username', 'course_id', 'action', 'week']).sum().reset_index()
new_test_prophet = new_test.groupby(by=['username', 'course_id', 'action', 'week']).sum().reset_index()

In [25]:
new_train_prophet_pivot = new_train_prophet.pivot(index=['username', 'course_id', 'action'], columns='week').fillna(0).reset_index()
new_test_prophet_pivot = new_test_prophet.pivot(index=['username', 'course_id', 'action'], columns='week').fillna(0).reset_index()

In [26]:
new_train_prophet_pivot

username                             course_id   action count         \
week                                                               1      2   
0             5                 TsinghuaX/60510102X/_    close   1.0    0.0   
1             5                 TsinghuaX/60510102X/_    video  10.0    0.0   
2             5           TsinghuaX/70240183x/2015_T2    close   3.0    0.0   
3             5           TsinghuaX/70240183x/2015_T2  problem   1.0    0.0   
4             5           TsinghuaX/70240183x/2015_T2    video  39.0    0.0   
...         ...                                   ...      ...   ...    ...   
452402  7023562  course-v1:TsinghuaX+AP000008X+2017T1  problem   0.0   10.0   
452403  7023562  course-v1:TsinghuaX+AP000008X+2017T1    video   0.0  228.0   
452404  7036067  course-v1:TsinghuaX+AP000008X+2017T1    click   0.0    0.0   
452405  7036067  course-v1:TsinghuaX+AP000008X+2017T1  problem   0.0    0.0   
452406  7036067  course-v1:TsinghuaX+AP000008X+2017T1    video   0.0    0.0   

                            
week      3    4    5    6  
0       0.0  0.0  0.0  0.0  
1       0.0  0.0  0.0  0.0  
2       0.0  0.0  0.0  0.0  
3       0.0  0.0  0.0  0.0  
4       0.0  0.0  0.0  0.0  
...     ...  ...  ...  ...  
452402  0.0  0.0  0.0  0.0  
452403  0.0  0.0  0.0  0.0  
452404  7.0  0.0  0.0  0.0  
452405  8.0  0.0  0.0  0.0  
452406  5.0  0.0  0.0  0.0  

[452407 rows x 9 columns]

In [27]:
new_test_prophet_pivot

username                              course_id   action count       \
week                                                                1    2   
0             5    course-v1:Tsinghua+20150001+2015_T2    click  14.0  0.0   
1             5    course-v1:Tsinghua+20150001+2015_T2    close   3.0  0.0   
2             5    course-v1:Tsinghua+20150001+2015_T2    video  12.0  0.0   
3             5  course-v1:TsinghuaX+00740123X+2016_T1    click   0.0  0.0   
4             5  course-v1:TsinghuaX+00740123X+2016_T1    video   0.0  0.0   
...         ...                                    ...      ...   ...  ...   
193808  7006106   course-v1:TsinghuaX+AP000008X+2017T1    video   0.0  9.0   
193809  7008105   course-v1:TsinghuaX+AP000008X+2017T1    click   1.0  0.0   
193810  7008105   course-v1:TsinghuaX+AP000008X+2017T1  problem   3.0  0.0   
193811  7008105   course-v1:TsinghuaX+AP000008X+2017T1    video   1.0  0.0   
193812  7059814   course-v1:TsinghuaX+AP000008X+2017T1  problem   0.0  0.0   

                             
week      3     4    5    6  
0       0.0   0.0  0.0  0.0  
1       0.0   0.0  0.0  0.0  
2       0.0   0.0  0.0  0.0  
3       0.0   0.0  2.0  0.0  
4       0.0   0.0  5.0  0.0  
...     ...   ...  ...  ...  
193808  0.0   0.0  0.0  0.0  
193809  0.0   0.0  0.0  0.0  
193810  0.0   0.0  0.0  0.0  
193811  0.0   0.0  0.0  0.0  
193812  0.0  95.0  0.0  0.0  

[193813 rows x 9 columns]

In [28]:
train_prophet = new_train_prophet_pivot.drop(columns=['username', 'course_id', 'action']).T.reset_index().drop(columns=['level_0'])
train_prophet.rename(columns={'week': 'ds'}, inplace=True)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [29]:
train_prophet

,ds,0,1,2,3,4,5,6,7,8,...,452397,452398,452399,452400,452401,452402,452403,452404,452405,452406
0,1,1.0,10.0,3.0,1.0,39.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,21.0,8.0,10.0,228.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,8.0,5.0
3,4,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,...,11.0,4.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,0.0,0.0,0.0,0.0,5.0,46.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Use Prophet library

In [242]:
from prophet import Prophet
from sklearn.metrics import mean_absolute_error

In [281]:
def prediction_prophet(userid, K, train_pivot):
    # select user to predict on
    train_pivot_copy = train_pivot.copy()
    train_pivot_copy.rename(columns={userid: 'y'}, inplace=True)
    all_users_in_course = [col for col in train_pivot_copy.columns if col != 'ds' and col != 'y']
    n_weeks = train_pivot_copy.shape[0]
    # change to date format for prophet to run
    train_pivot_copy['ds'] = pd.date_range(start="2022-01-01",end="2022-01-06") # these are dummy dates
    
    # create lists to be saved for later
    models = []
    y_true_running = []
    y_pred_running = []
    maes = []
    
    for t in range(2, n_weeks-1):
        # reduce training set to first t weeks
        train_pivot_t = train_pivot_copy.head(t)
        test_pivot_t = train_pivot_copy.tail(-t)
        
        train_pivot_t.columns = train_pivot_t.columns.astype(str)
        
        # create the model with K additional regressors (other students)
        model = Prophet()
        for other_userid in all_users_in_course[:K]:
            model.add_regressor(str(other_userid))
            
        print(train_pivot_t)
        
        # fit the model
        model.fit(train_pivot_t)
        models.append(model)
        
        # perform forecast
        future = model.make_future_dataframe(periods=test_pivot_t.shape[0])
        columns_to_keep = train_pivot_copy.columns[2:K+2]
        for_future = train_pivot_copy[columns_to_keep]
        futures = pd.concat((future, for_future), axis=1)
        forecast = model.predict(futures)
        
        # log results
        forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
        y_true = test_pivot_t['y'].values
        y_true_running.append(y_true)
        y_pred = forecast['yhat'][t:].values
        y_pred_running.append(y_pred)
        mae = mean_absolute_error(y_true, y_pred)
        maes.append(mae)
    
    return models, y_true_running, y_pred_running, maes

In [ ]:
maes_dict = {}
for userid in list(train_prophet.columns[1:]):
    print(userid)
    maes_list = []
    print(train_prophet.shape[1]-2)
    models, y_true_running, y_pred_running, maes = prediction_prophet(userid=userid, K=train_prophet.shape[1]-2, \
                                                                      train_pivot=train_prophet)
    maes_dict[userid] = maes

0
452410
          ds    y     1    2    3     4    5    6    7    8  ...  452401  \
0 2022-01-01  1.0  10.0  3.0  1.0  39.0  0.0  0.0  0.0  0.0  ...     0.0   
1 2022-01-02  0.0   0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  ...     0.0   

   452402  452403  452404  452405  452406  452407  452408  452409  452410  
0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0  
1     0.0     0.0    21.0     8.0    10.0   228.0     0.0     0.0     0.0  

[2 rows x 452412 columns]


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
